In [1]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [16]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import tidytcells as tt

from src.model_loader import ModelLoader

In [5]:
def load_decombinator_csv_from_path(p: Path) -> DataFrame:
    raw_decombinator_table = pd.read_csv(p)
    cleaned_decombinator_table = standardize_tcr_nomenclature(raw_decombinator_table)
    cleaned_decombinator_table = cleaned_decombinator_table[["v_call", "junction_aa", "j_call"]]

    if is_alpha_chain_data(cleaned_decombinator_table):
        cleaned_decombinator_table.columns = ["TRAV", "CDR3A", "TRAJ"]
    else:
        cleaned_decombinator_table.columns = ["TRBV", "CDR3B", "TRBJ"]

    return cleaned_decombinator_table

def standardize_tcr_nomenclature(decombinator_table: DataFrame) -> DataFrame:
    decombinator_table.v_call = decombinator_table.v_call.map(standardize_trb_gene_and_only_keep_functional)
    decombinator_table.j_call = decombinator_table.j_call.map(standardize_trb_gene_and_only_keep_functional)
    decombinator_table.junction_aa = decombinator_table.junction_aa.map(tt.junction.standardize)

    return decombinator_table

def standardize_trb_gene_and_only_keep_functional(gene: str) -> str:
    return tt.tcr.standardize(gene=gene, enforce_functional=True)
    
def is_alpha_chain_data(decombinator_table: DataFrame) -> bool:
    first_v_gene = decombinator_table.v_call.iloc[0]
    first_v_gene_is_trav = "TRAV" in first_v_gene
    
    return first_v_gene_is_trav

In [8]:
alpha_df = load_decombinator_csv_from_path("/home/yutanagano/Downloads/TST_2023/alpha_TCR_all.csv")
beta_df = load_decombinator_csv_from_path("/home/yutanagano/Downloads/TST_2023/beta_TCR_all.csv")

/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV8-7" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV8-7".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAJ58" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAJ58".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAJ25" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAJ25".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAJ61" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAJ61".
  warn(
/home/yutanagano/Projects/tcr_

In [9]:
beta_only_model = ModelLoader(Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/model_saves/BCDRBERT_ACL_small_double_censoring"))
paired_chain_model = ModelLoader(Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/model_saves/CDRBERT_+ACL_cns_020_cdrcns_001_chaincns_020"))

In [10]:
beta_embeddings = beta_only_model.embed(beta_df)

/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:296: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


In [17]:
np.save("beta_embeddings.npy", beta_embeddings)

In [18]:
alpha_embeddings = paired_chain_model.embed(alpha_df)

In [21]:
np.save("alpha_embeddings.npy", alpha_embeddings)